# __Analyzing CIA Factbook Data Using SQL__

In this SQL project we will explore the [CIA World Factbook]( https://www.cia.gov/the-world-factbook/), a compendium of statistics about all countries on Earth such as land mass and population. 

To use SQL within Jupyter Notebook we first need to install [ipython-sql]( https://github.com/catherinedevlin/ipython-sql), if you have the Anaconda installed you can do so by simply running [this code]( https://anaconda.org/conda-forge/ipython-sql)

In [1]:
#!conda install -yc conda-forge ipython-sql

# __Importing Database__

To connect our Jupyter Notebook to our database we use `%%capture`, `%load_ext sql` and `sqlite:///` 

In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

To run our SQL queries, we need to add `%%sql` at the start of each query. To check that our database is properly connected to our Jupyter Notebook we will create a SELECT query from sqlite_master to list all tables. 

In [3]:
%%sql
SELECT *
FROM sqlite_master
WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


We can see that our __factbook.db__ has been connected and created a database called __facts__.  We can now look at the first couple of rows to analyses columns in our database. 

In [4]:
%%sql 
SELECT * FROM facts LIMIT 5

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


We can see that our database has 11 columns 
- `id`: A unique number for each row
- `code`: The unique 2 letter code for each country 
- `name`: The name of the country 
- `area`: The country’s area in square kilometers 
- `area_land`: The country’s land area in square kilometers
- `area_water`: The country’s water area in square kilometers
- `population`: The country’s population
- `population_growth`: The country's population growth as a percentage.
- `birth_rate`: The country's birth rate, or the number of births a year per 1,000 people.
- `death_rate`: The country's death rate, or the number of death a year per 1,000 people.
- `migration_rate`: The country's migration rate, or the number of migrants a year per 1,000 people.

# __Summarizing Statistics__

The next step is for us to use SQL to explore some summarizing statistics and search for some potential outliners. We will start with looking at population and population growth using the [`MIN()`, `Max()` and AVG()](https://www.sqlite.org/lang_corefunc.html) functions. 

In [5]:
%%sql 
SELECT MIN(population) AS Smallest_Population, 
MAX(population) AS Largest_Population,
AVG(population) AS Average_Population,
MIN(Area) AS Smallest_Area,
MAX(Area) AS Largest_Area,
AVG(Area) AS Average_Area
FROM facts

 * sqlite:///factbook.db
Done.


Smallest_Population,Largest_Population,Average_Population,Smallest_Area,Largest_Area,Average_Area
0,7256490011,62094928.32231405,0,17098242,555093.546184739


We can see that there are some strange outliners in our summarizing statistics:
- There is a country with a population of `0`
- There is a country with a area of `0`
- There is a country with a population of `7256490011` 
- The average population per country is `62094928`

We can create a SQL query that determines the name of our outliers where we insert our original query in our WHERE clause. 

### __Finding Countries with Populations and Area of `0`__

In [6]:
%%sql 
SELECT *
FROM facts 
WHERE population == (SELECT MIN(population)
                    FROM facts)

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


We can see that Antarctica is the country with a population and area of `0`. This makes sense as Antarctica has no indigenous inhabits which is explained in the [factbook]( https://www.cia.gov/the-world-factbook/countries/antarctica/) 

### __Finding Countries with Populations of `7256490011`__

In [7]:
%%sql 
SELECT *
FROM facts 
WHERE population == (SELECT MAX(population)
                    FROM facts)

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


We can see that the database also contains a row containing values on the entire world. This makes sense as 7.2 billion we got before represents the entire human population. This also explain why our average population is over 62 million per country as that value appears to be too large. We can exclude the `World` row in our future statistics by add in name != ‘World’ to our WHERE clause. 

### __Finding other special rows__

Now that we know that the data contains special columns like `World` that contains the entire world population, we need to see if there are any other columns like this that could affect our statistics. We can create a query that will order the rows by population. 

In [8]:
%%sql 
SELECT name, population, population_growth
FROM facts
ORDER BY population DESC

 * sqlite:///factbook.db
Done.


name,population,population_growth
World,7256490011,1.08
China,1367485388,0.45
India,1251695584,1.22
European Union,513949445,0.25
United States,321368864,0.78
Indonesia,255993674,0.92
Brazil,204259812,0.77
Pakistan,199085847,1.46
Nigeria,181562056,2.45
Bangladesh,168957745,1.6


We can see in our output that on the top of our list we see the European Union which is a collective of 27 countries which is something to exclude as these countries are already accounted for. On the other end we can see islands such as the Wake Island and McDonald Island which are uninhabited, we also see entries for Oceans such as the Pacific and Atlantic. As these entries have their numeric vales set as `None` SQLite will exclude them automatically in any calculation.  

## __Cleaned Summarizing Statistics__

In [9]:
%%sql 
SELECT MIN(population) AS Smallest_Population, 
MAX(population) AS Largest_Population,
AVG(population) AS Average_Population,
MIN(Area) AS Smallest_Area,
MAX(Area) AS Largest_Area,
AVG(Area) AS Average_Area
FROM facts
WHERE name NOT IN ('World', 'European Union')

 * sqlite:///factbook.db
Done.


Smallest_Population,Largest_Population,Average_Population,Smallest_Area,Largest_Area,Average_Area
0,1367485388,30235554.991666667,0,17098242,539893.1895161291


We can see now that the Largest_Population is now 1.3 billion which is much more realistic than 7.2. we can also see that the Average_Population is now 30 million compared to 62 million we saw before. 

## __Finding countries with Largest Population and Growrh_rate__

Now that we know that we have to exclude the World-wide population we can see what countries have the hugest population and population_growth. 

In [10]:
%%sql 

SELECT name, MAX(population) AS 'Population'
FROM facts
WHERE name NOT IN ('World', 'European Union')

 * sqlite:///factbook.db
Done.


name,Population
China,1367485388


In [11]:
%%sql 

SELECT name, population, MAX(population_growth) AS 'Growth Rate'
FROM facts
WHERE name NOT IN ('World', 'European Union')

 * sqlite:///factbook.db
Done.


name,population,Growth Rate
South Sudan,12042910,4.02


We can see that China has the largest population with 1.3 billion which is what we saw in our summarizing statistics earlier. The country with the highest population growth rate is the South Sudan. 

## __What Countries Will Have the Biggest Population Increase__

One thing we can predict with our data is seeing which country will have the biggest increase in population in the next year using the population column and the population_growth column. To see how mush the population would increase within a year we can multiply the current population with the percentage it would increase.

In [12]:
%%sql

SELECT name, 
ROUND((population * (population_growth/100))) AS Pop_increase
FROM facts
WHERE name NOT IN ('World', 'European Union')
ORDER BY Pop_increase DESC
LIMIT 10

 * sqlite:///factbook.db
Done.


name,Pop_increase
India,15270686.0
China,6153684.0
Nigeria,4448270.0
Pakistan,2906653.0
Ethiopia,2874562.0
Bangladesh,2703324.0
United States,2506677.0
Indonesia,2355142.0
"Congo, Democratic Republic of the",1944691.0
Philippines,1626074.0


We can see that India will have the highest population increase with 15 million new citizens. We also see that China is in second place in population increase even though it has the highest population. We also don’t see South Sudan in the top ten even though it had the highest population_growth, this is due to South Sudan having a smaller population so they have a smaller population increase. It is also interesting to note that within the top ten countries 6 are in Asia, 3 are in Africa and 1 is in North America. 

## __What Countries have a Larger Death Rate than Birth Rate__

Another aspect of the database we can look at is comparing countries birth_rate to their own death_rate. We can accomplish this by adding the > operator on the death_rate and birth_rate column on the WHERE clause.

In [13]:
%%sql 
SELECT name, 
birth_rate,
death_rate
FROM facts
WHERE death_rate > birth_rate

 * sqlite:///factbook.db
Done.


name,birth_rate,death_rate
Austria,9.41,9.42
Belarus,10.7,13.36
Bosnia and Herzegovina,8.87,9.75
Bulgaria,8.92,14.44
Croatia,9.45,12.18
Czech Republic,9.63,10.34
Estonia,10.51,12.4
Germany,8.47,11.42
Greece,8.66,11.09
Hungary,9.16,12.73


We can see in our output above that there are 24 countries where the death_rate outpaces the birth_rate. It is interesting to note that 23 of the countries with a higher death_rate than birth_rate are in Europe. 

## __Finding Countries with Above Average Population and Area__

Now that we have our cleaned values for the average population and area, we can find countries which have higher than average population and area.

we can do this by creating a query where we select all columns and adding 2 subqueries to our where clause to filter above the average values. 

In [14]:
%%sql 
SELECT *
FROM facts
WHERE population > (SELECT AVG(population) 
                    FROM facts
                    WHERE name NOT IN ('World', 'European Union'))
                AND area > (SELECT AVG(area)
                            FROM facts)
                AND name NOT IN ('World', 'European Union')

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
7,ar,Argentina,2780400,2736690,43710,43431886,0.93,16.64,7.33,0.0
24,br,Brazil,8515770,8358140,157630,204259812,0.77,14.46,6.58,0.14
28,bm,Burma,676578,653508,23070,56320206,1.01,18.39,7.96,0.28
32,ca,Canada,9984670,9093507,891163,35099836,0.75,10.28,8.42,5.66
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44
38,co,Colombia,1138910,1038700,100210,46736728,1.04,16.47,5.4,0.64
40,cg,"Congo, Democratic Republic of the",2344858,2267048,77810,79375136,2.45,34.88,10.07,0.27
53,eg,Egypt,1001450,995450,6000,88487396,1.79,22.9,4.77,0.19


We can see that our query has returned 27 countries all of which have a population and area greater than the global average. 

## __What Countries have More Water Than Land__

As the database contains both countries land and water area in the columns area_land and area_water, we can look at what countries have more water than land mass. We can determine this by dividing the area_water by area_land, if the result is above 1 then there is more water than land. When we do this we need to use the CAST() function to convert both of the columns to Floats, otherwise our results will be rounded to the nearest integer making it impossible to compare the ratio of water to land.

In [15]:
%%sql 
SELECT name, area_water, area_land,
ROUND(CAST(area_water AS Float) / CAST(area_land AS Float), 2) as Water_Land_Ratio
FROM facts
ORDER BY Water_Land_Ratio DESC
LIMIT 10

 * sqlite:///factbook.db
Done.


name,area_water,area_land,Water_Land_Ratio
British Indian Ocean Territory,54340,60,905.67
Virgin Islands,1564,346,4.52
Puerto Rico,4921,8870,0.55
"Bahamas, The",3870,10010,0.39
Guinea-Bissau,8005,28120,0.28
Malawi,24404,94080,0.26
Netherlands,7650,33893,0.23
Uganda,43938,197100,0.22
Eritrea,16600,101000,0.16
Liberia,15049,96320,0.16


We can see there are only 2 countries with more water than land, `British Indian Ocean Territory` and ` Virgin Islands`. This makes sense as these two countries are island nations with oceans around them. We also see other island nations in the top ten such as `Puerto Rico` and `The Bahamas` with ratios of 0.55 and 0.39 respectively. We also see countries with large coast lines such as Guinea-Bissau and Netherlands. Surprisingly, we also see landlocked countries on this list such as `Uganda` and `Malawi` due to Lake Victoria and Lake Malawi which are large freshwater lakes.

## __Population to Area__

In this last part of our analysis, we will compare a countries population to area. Before we looked at countries that have an above average population and area and we can compare these two outputs. To calculate the population to are ratio we can divide the country’s population column by the country’s area. Again we will use the ROUND() and CAST() functions to convert the values to Floats. 

In [16]:
%%sql
SELECT name, population, area,
Round(CAST(population AS Float) / CAST(area AS Float),1) AS Pop_Area_Ratio
FROM facts 
WHERE name NOT IN ('World', 'European Union')
ORDER BY Pop_Area_Ratio DESC
LIMIT 10

 * sqlite:///factbook.db
Done.


name,population,area,Pop_Area_Ratio
Macau,592731,28,21169.0
Monaco,30535,2,15267.5
Singapore,5674472,697,8141.3
Hong Kong,7141106,1108,6445.0
Gaza Strip,1869055,360,5191.8
Gibraltar,29258,6,4876.3
Bahrain,1346613,760,1771.9
Maldives,393253,298,1319.6
Malta,413965,316,1310.0
Bermuda,70196,54,1299.9


We can see that the countries with the highest `Pop_Area_Ratio` are incredibly small countries with large cities such as Macau, Monaco and Singapore. We can now compare this to our earlier query where we looked for countries that have an above average population and area. This time however we will add an `ORDER BY` clause to order by both the population first and then by area and we will add our `Pop_Area_Ratio` column to compare our tables.   

### __Pop_Area_Ratio Compared to Population and Area__

#### __Ordering Countries by Population__

In [17]:
%%sql 
SELECT name, population, area,
Round(CAST(population AS Float) / CAST(area AS Float),1) AS Pop_Area_Ratio
FROM facts
WHERE population > (SELECT AVG(population) 
                    FROM facts
                    WHERE name NOT IN ('World', 'European Union'))
    AND area > (SELECT AVG(area)
                FROM facts)
    AND name NOT IN ('World', 'European Union')
ORDER BY population DESC
LIMIT 10

 * sqlite:///factbook.db
Done.


name,population,area,Pop_Area_Ratio
China,1367485388,9596960,142.5
India,1251695584,3287263,380.8
United States,321368864,9826675,32.7
Indonesia,255993674,1904569,134.4
Brazil,204259812,8515770,24.0
Pakistan,199085847,796095,250.1
Nigeria,181562056,923768,196.5
Russia,142423773,17098242,8.3
Mexico,121736809,1964375,62.0
Ethiopia,99465819,1104300,90.1


#### __Ordering Countries by Area__

In [18]:
%%sql 
SELECT name, population, area,
Round(CAST(population AS Float) / CAST(area AS Float),1) AS Pop_Area_Ratio
FROM facts
WHERE population > (SELECT AVG(population) 
                    FROM facts
                    WHERE name != 'World' OR 'European Union')
      AND area > (SELECT AVG(area)
                            FROM facts)
      AND name NOT IN ('World', 'European Union')
ORDER BY area DESC
LIMIT 10

 * sqlite:///factbook.db
Done.


name,population,area,Pop_Area_Ratio
Russia,142423773,17098242,8.3
Canada,35099836,9984670,3.5
United States,321368864,9826675,32.7
China,1367485388,9596960,142.5
Brazil,204259812,8515770,24.0
India,1251695584,3287263,380.8
Argentina,43431886,2780400,15.6
Algeria,39542166,2381741,16.6
"Congo, Democratic Republic of the",79375136,2344858,33.9
Mexico,121736809,1964375,62.0


We can see that none of the rows from ` Pop_Area_Ratio` appear in our population of area table. China which is country with the highest population has a population/area ratio almost 150 smaller than its neighboring county Macau. This is the same case for Russia which has the largest area, Russia’s population/area is 2000 times smaller than Macau. 

# __Conclusion__

In this project we were able to import our FactBook database and analyse it by successfully running SQL in Jupyter Notebook.  Using SQL, we where able to determine several things about worldwide population.
- We found that India will have the largest population increase within the next year, Increasing its population by 15 million 
- We found that several countries in Europe and Japan have a higher `death_rate` than `birth_rate`.
- Only 2 countries have more water area than land area and some landlocked countries have high water area.
- Countries with high population and area have a small population to area ratio. 

A way to expand this analysis in the future would be to include more data from the CIA Factbook and look at other aspects like `Land_Use`, `Natural_Resoutces`, `Languages`, `Religion`.